In [1]:
#!pip install qiskit.extensions
from qiskit import *
from qiskit_aer import Aer

# Reduced Simon Cipher Implementation

In [23]:
#key schedule function

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

rounds = 4
m = 2
n = 3

round_keys = [[0 for _ in range(n)] for _ in range(m)]

key = [1,1,0,0,0,1]

constant = [0,0,1]

plaintext = [0,1,1,1,0,1]

left = QuantumRegister(n,name = "left")
right = QuantumRegister(n, name = "right")
ciphertext = ClassicalRegister(6, name = "ciphertext")

qc = QuantumCircuit(left, right, ciphertext)

for i in range(3):
    if plaintext[i] == 1:
        qc.x(left[i])

for i in range(3):
    if plaintext[3 + i] == 1:
        qc.x(right[i])


for i in range(rounds):

    key_index = i%m

    if i < m:
        for j in range(0,n):
            round_keys[key_index][j] = key[n*(m - i - 1) + j]

    else:
        print("===============")
        print(round_keys[key_index])
        for j in range(n):
            round_keys[key_index][j] ^= constant[j] ^ round_keys[key_index - 1][j - 1] ^ round_keys[key_index - 1][j - 2]
            print(round_keys[key_index])
        print("===============")

    print(round_keys[key_index])
    
    if i%2 == 0:
        for j in range(0,n):
            qc.ccx(left[(j + 1)%n], left[(j + 2)%n], right[j])
        for j in range(0,n):
            qc.cx(left[(j)%n], right[j])
        for j in range(0,n):
            if round_keys[key_index][j] == 1:
                qc.x(right[j])
    else:
        for j in range(0,n):
            qc.ccx(right[(j + 1)%n], right[(j + 2)%n], left[j])
        for j in range(0,n):
            qc.cx(right[(j)%n], left[j])
        for j in range(0,n):
            if round_keys[key_index][j] == 1:
                qc.x(left[j])    

ct = 0
for i in range(n):
    qc.measure(left[i], ciphertext[i])

ct = 3
for i in range(n):
    qc.measure(right[i], ciphertext[ct + i])

[0, 0, 1]
[1, 1, 0]
[0, 0, 1]
[1, 0, 1]
[1, 1, 1]
[1, 1, 0]
[1, 1, 0]
[1, 1, 0]
[0, 1, 0]
[0, 0, 0]
[0, 0, 1]
[0, 0, 1]


In [17]:
def count_gates(circuit):
    cnot_count = circuit.count_ops().get('cx', 0)
    x_count = circuit.count_ops().get('x', 0)
    toffoli_count = circuit.count_ops().get('ccx', 0)
    print("CNOT gates :", cnot_count)
    print("X gates :", x_count)
    print("Toffoli gates :", toffoli_count)
    print("Depth :", qc.depth())
    print("qubits :",len(qc.qubits))
    return cnot_count, x_count, toffoli_count

a,b,c = count_gates(qc)

print(qc)

CNOT gates : 12
X gates : 10
Toffoli gates : 12
Depth : 22
qubits : 6
                                                      ┌───┐          ┌───┐┌───┐»
      left_0: ────────────■────■────■─────────────────┤ X ├──────────┤ X ├┤ X ├»
              ┌───┐       │    │    │                 └─┬─┘┌───┐     └─┬─┘├───┤»
      left_1: ┤ X ├──■────┼────■────┼────■──────────────┼──┤ X ├───────┼──┤ X ├»
              ├───┤  │    │    │    │    │              │  └─┬─┘┌───┐  │  └─┬─┘»
      left_2: ┤ X ├──■────■────┼────┼────┼────■─────────┼────┼──┤ X ├──┼────┼──»
              ├───┤┌─┴─┐  │    │  ┌─┴─┐  │    │         │    │  └─┬─┘  │    │  »
     right_0: ┤ X ├┤ X ├──┼────┼──┤ X ├──┼────┼─────────┼────■────■────■────┼──»
              └───┘└───┘┌─┴─┐  │  └───┘┌─┴─┐  │         │    │    │         │  »
     right_1: ──────────┤ X ├──┼───────┤ X ├──┼─────────■────┼────■─────────■──»
              ┌───┐     └───┘┌─┴─┐     └───┘┌─┴─┐┌───┐  │    │                 »
     right_2: ┤ X ├──────────┤ X ├─────

In [18]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer

backend = Aer.get_backend('qasm_simulator')

# Run the circuit on the qasm simulator for a single shot
result = backend.run(qc, shots=1).result()

# Get the counts for the single shot
counts = result.get_counts()
print(counts)

{'111110': 1}


# Correct implementation of forrelation circuit

In [19]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

rounds = 4
m = 2
n = 3

round_keys = [[0 for _ in range(n)] for _ in range(m)]

key = [1,1,0,0,0,1]

constant = [0,0,1]

plaintext = [0,1,1,1,0,1]

left1 = QuantumRegister(n,name = "left1")
right1 = QuantumRegister(n, name = "right1")
ancilla1 = QuantumRegister(1, name = "ancilla1")

ciphertext = ClassicalRegister(2*n, name = "ciphertext")

qc = QuantumCircuit(left1, right1, ancilla1, ciphertext)

# for i in range(3):
#     if plaintext[i] == 1:
#         qc.x(left1[i])
#         qc.x(left2[i])

# for i in range(3):
#     if plaintext[3 + i] == 1:
#         qc.x(right1[i])
#         qc.x(right2[i])

rounds = 4
m = 2
n = 3

round_keys = [[0 for _ in range(n)] for _ in range(m)]

key = [1,1,0,0,0,1]

constant = [0,0,1]

for i in range(rounds):

    key_index = i%m

    if i < m:
        for j in range(0,n):
            round_keys[key_index][j] = key[n*(m - i - 1) + j]

    else:
        # print("===============")
        # print(round_keys[key_index])
        for j in range(n):
            round_keys[key_index][j] ^= constant[j] ^ round_keys[key_index - 1][j - 1] ^ round_keys[key_index - 1][j - 2]
        #     print(round_keys[key_index])
        # print("===============")

    #print(round_keys[key_index])
    
    if i%2 == 0:
        for j in range(0,n):
            qc.ccx(left1[(j + 1)%n], left1[(j + 2)%n], right1[j])
        for j in range(0,n):
            qc.cx(left1[(j)%n], right1[j])
        for j in range(0,n):
            if round_keys[key_index][j] == 1:
                qc.x(right1[j])
    else:
        for j in range(0,n):
            qc.ccx(right1[(j + 1)%n], right1[(j + 2)%n], left1[j])
        for j in range(0,n):
            qc.cx(right1[(j)%n], left1[j])
        for j in range(0,n):
            if round_keys[key_index][j] == 1:
                qc.x(left1[j])    

##########################
simon_dagger = qc.inverse()
simon = qc.copy()
simon_dagger2 = simon_dagger.copy()

##########################
qc.cx(left1[0], ancilla1[0])
simon.cx(left1[1], ancilla1[0])

##########creating oracles###########
oracle1 = qc.compose(simon_dagger)
oracle2 = simon.compose(simon_dagger2)

left1 = QuantumRegister(n,name = "left1")
right1 = QuantumRegister(n, name = "right1")
ancilla1 = QuantumRegister(1, name = "ancilla1")
ancilla2 = QuantumRegister(1, name = "ancilla2")
ciphertext = ClassicalRegister(2*n, name = "ciphertext")
final = QuantumCircuit(left1, right1, ancilla1, ancilla2, ciphertext)


final.x(ancilla1[0])
final.h(ancilla1[0])
final.barrier

for i in range(3):
    final.h(left1[i])
    final.h(right1[i])

final.barrier()

final = final.compose(oracle1)

final.barrier()

for i in range(3):
    final.h(left1[i])
    final.h(right1[i])

final.barrier()

final = final.compose(oracle2)

final.barrier()

for i in range(3):
    final.h(left1[i])
    final.h(right1[i])

qc.barrier()

for i in range(3):
    final.measure(left1[i],ciphertext[i])
    final.measure(right1[i],ciphertext[i+3])

print(final)

              ┌───┐      ░                                    ┌───┐          »
     left1_0: ┤ H ├──────░────────■────■────■─────────────────┤ X ├──────────»
              ├───┤      ░        │    │    │                 └─┬─┘┌───┐     »
     left1_1: ┤ H ├──────░───■────┼────■────┼────■──────────────┼──┤ X ├─────»
              ├───┤      ░   │    │    │    │    │              │  └─┬─┘┌───┐»
     left1_2: ┤ H ├──────░───■────■────┼────┼────┼────■─────────┼────┼──┤ X ├»
              ├───┤      ░ ┌─┴─┐  │    │  ┌─┴─┐  │    │         │    │  └─┬─┘»
    right1_0: ┤ H ├──────░─┤ X ├──┼────┼──┤ X ├──┼────┼─────────┼────■────■──»
              ├───┤      ░ └───┘┌─┴─┐  │  └───┘┌─┴─┐  │         │    │    │  »
    right1_1: ┤ H ├──────░──────┤ X ├──┼───────┤ X ├──┼─────────■────┼────■──»
              ├───┤      ░      └───┘┌─┴─┐     └───┘┌─┴─┐┌───┐  │    │       »
    right1_2: ┤ H ├──────░───────────┤ X ├──────────┤ X ├┤ X ├──■────■───────»
              ├───┤┌───┐ ░           └───┘          

In [20]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')
result = backend.run(final, shots=10000).result()
counts = result.get_counts()
print((counts['000000']/10000)**(0.5))
print(counts)

0.07745966692414834
{'111010': 54, '100110': 410, '001111': 64, '111111': 46, '001010': 128, '100001': 545, '100111': 732, '111000': 378, '101100': 60, '000011': 26, '110100': 517, '010011': 576, '001011': 24, '100010': 56, '111110': 571, '001000': 72, '011101': 442, '010010': 301, '110101': 4, '000010': 398, '101011': 580, '111001': 50, '000000': 60, '110111': 303, '000110': 3, '101111': 308, '110001': 119, '010000': 392, '110000': 63, '000111': 383, '000001': 47, '101010': 24, '110110': 368, '010100': 297, '000100': 1, '101101': 70, '110011': 304, '000101': 27, '101110': 20, '110010': 196, '010101': 27, '111101': 56, '100011': 74, '001100': 122, '100101': 16, '111011': 23, '001110': 73, '011010': 55, '101000': 73, '011001': 131, '001101': 2, '100100': 193, '010110': 21, '011011': 30, '011110': 54, '011000': 3, '010111': 17, '010001': 4, '101001': 1, '001001': 2, '011100': 2, '011111': 1, '100000': 1}


In [21]:
def  generate_circuit(first,second):
    rounds = 4
    m = 2
    n = 3

    round_keys = [[0 for _ in range(n)] for _ in range(m)]

    key = [1,1,0,0,0,1]

    constant = [0,0,1]

    plaintext = [0,1,1,1,0,1]

    left1 = QuantumRegister(n, name = "left1")
    right1 = QuantumRegister(n, name = "right1")
    ancilla1 = QuantumRegister(1, name = "ancilla1")

    ciphertext = ClassicalRegister(2*n, name = "ciphertext")

    qc = QuantumCircuit(left1, right1, ancilla1, ciphertext)

    rounds = 4
    m = 2
    n = 3

    round_keys = [[0 for _ in range(n)] for _ in range(m)]

    key = [1,1,0,0,0,1]

    constant = [0,0,1]

    for i in range(rounds):

        key_index = i%m

        if i < m:
            for j in range(0,n):
                round_keys[key_index][j] = key[n*(m - i - 1) + j]

        else:
            # print("===============")
            # print(round_keys[key_index])
            for j in range(n):
                round_keys[key_index][j] ^= constant[j] ^ round_keys[key_index - 1][j - 1] ^ round_keys[key_index - 1][j - 2]
            #     print(round_keys[key_index])
            # print("===============")

        #print(round_keys[key_index])
        
        if i%2 == 0:
            for j in range(0,n):
                qc.ccx(left1[(j + 1)%n], left1[(j + 2)%n], right1[j])
            for j in range(0,n):
                qc.cx(left1[(j)%n], right1[j])
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    qc.x(right1[j])
        else:
            for j in range(0,n):
                qc.ccx(right1[(j + 1)%n], right1[(j + 2)%n], left1[j])
            for j in range(0,n):
                qc.cx(right1[(j)%n], left1[j])
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    qc.x(left1[j])    

    ##########################
    simon_dagger = qc.inverse()
    simon = qc.copy()
    simon_dagger2 = simon_dagger.copy()

    ##########################

    if first >= n:
        first = first - n
        qc.cx(right1[first], ancilla1[0])
    else :
        qc.cx(left1[first], ancilla1[0])
    
    
    if second >= n:
        second = second - n
        simon.cx(right1[second], ancilla1[0])
    else :
        simon.cx(left1[second], ancilla1[0])

    # qc.cx(left1[first], ancilla1[0])
    # simon.cx(left1[second], ancilla1[0])

    ##########creating oracles###########
    oracle1 = qc.compose(simon_dagger)
    oracle2 = simon.compose(simon_dagger2)

    left1 = QuantumRegister(n,name = "left1")
    right1 = QuantumRegister(n, name = "right1")
    ancilla1 = QuantumRegister(1, name = "ancilla1")
    ancilla2 = QuantumRegister(1, name = "ancilla2")
    ciphertext = ClassicalRegister(2*n, name = "ciphertext")
    final = QuantumCircuit(left1, right1, ancilla1, ancilla2, ciphertext)


    final.x(ancilla1[0])
    final.h(ancilla1[0])
    final.barrier

    for i in range(3):
        final.h(left1[i])
        final.h(right1[i])

    final.barrier()

    final = final.compose(oracle1)

    final.barrier()

    for i in range(3):
        final.h(left1[i])
        final.h(right1[i])

    final.barrier()

    final = final.compose(oracle2)

    final.barrier()

    for i in range(3):
        final.h(left1[i])
        final.h(right1[i])

    qc.barrier()

    for i in range(3):
        final.measure(left1[i],ciphertext[i])
        final.measure(right1[i],ciphertext[i+3])

    return final

In [22]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
for i in range(6):
    for j in range(i,6):
        print(i , j)
        final = generate_circuit(i,j)
        #print(final)
        backend = Aer.get_backend('qasm_simulator')
        result = backend.run(final, shots=10000).result()
        counts = result.get_counts()
        try:
            print((counts['000000']/10000)**(0.5))
        except KeyError:
            print(counts)
        #print(counts)
# final = generate_circuit(0,0)
# print(final)
# backend = Aer.get_backend('qasm_simulator')
# result = backend.run(final, shots=10000).result()
# counts = result.get_counts()
# print(counts['000000']/10000)

0 0
0.07874007874011811
0 1
0.07416198487095663
0 2
0.0469041575982343
0 3
0.03
0 4
0.03162277660168379
0 5
0.08717797887081347
1 1
0.07810249675906655
1 2
0.05
1 3
0.017320508075688773
1 4
0.034641016151377546
1 5
0.09
2 2
0.01
2 3
0.03
2 4
0.026457513110645904
2 5
0.0938083151964686
3 3
{'001010': 156, '100001': 989, '111111': 336, '010111': 37, '001111': 38, '111010': 620, '100110': 334, '011010': 328, '110111': 659, '111000': 337, '010010': 153, '100011': 146, '111101': 147, '001100': 603, '000010': 47, '110101': 53, '101011': 365, '110010': 354, '101110': 147, '000101': 370, '101001': 172, '011001': 137, '111100': 149, '100100': 136, '001101': 190, '100111': 364, '111110': 40, '001011': 160, '011111': 134, '011000': 156, '110000': 156, '101101': 33, '000100': 44, '110011': 35, '111011': 150, '001110': 42, '100101': 167, '010101': 183, '011011': 35, '110100': 32, '000011': 48, '101100': 40, '010110': 33, '010001': 38, '100000': 347, '010011': 158, '001000': 41, '000110': 166, '1011

In [23]:
def  generate_circuit(first,second):
    rounds = 4
    m = 2
    n = 3

    round_keys = [[0 for _ in range(n)] for _ in range(m)]

    key = [1,1,0,0,0,1]

    constant = [0,0,1]

    plaintext = [0,1,1,1,0,1]


    left1 = QuantumRegister(n,name = "left1")
    right1 = QuantumRegister(n, name = "right1")
    ancilla1 = QuantumRegister(1, name = "ancilla1")

    ciphertext = ClassicalRegister(2*n, name = "ciphertext")

    qc = QuantumCircuit(left1, right1, ancilla1, ciphertext)
    

    rounds = 4
    m = 2
    n = 3

    round_keys = [[0 for _ in range(n)] for _ in range(m)]

    key = [1,1,0,0,0,1]

    constant = [0,0,1]

    for i in range(rounds):

        key_index = i%m

        if i < m:
            for j in range(0,n):
                round_keys[key_index][j] = key[n*(m - i - 1) + j]

        else:
            # print("===============")
            # print(round_keys[key_index])
            for j in range(n):
                round_keys[key_index][j] ^= constant[j] ^ round_keys[key_index - 1][j - 1] ^ round_keys[key_index - 1][j - 2]
            #     print(round_keys[key_index])
            # print("===============")

        #print(round_keys[key_index])
        
        if i%2 == 0:
            for j in range(0,n):
                qc.ccx(left1[(j + 1)%n], left1[(j + 2)%n], right1[j])
            for j in range(0,n):
                qc.cx(left1[(j)%n], right1[j])
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    qc.x(right1[j])
        else:
            for j in range(0,n):
                qc.ccx(right1[(j + 1)%n], right1[(j + 2)%n], left1[j])
            for j in range(0,n):
                qc.cx(right1[(j)%n], left1[j])
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    qc.x(left1[j])    

    ##########################
    simon_dagger = qc.inverse()
    ##########################

    if first >= n:
        first = first - n
        qc.cx(right1[first], ancilla1[0])
    else :
        qc.cx(left1[first], ancilla1[0])

    # qc.cx(left1[first], ancilla1[0])
    # simon.cx(left1[second], ancilla1[0])

    ##########creating oracles###########
    oracle1 = qc.compose(simon_dagger)

    left1 = QuantumRegister(n,name = "left1")
    right1 = QuantumRegister(n, name = "right1")
    ancilla1 = QuantumRegister(1, name = "ancilla1")
    ancilla2 = QuantumRegister(1, name = "ancilla2")
    ciphertext = ClassicalRegister(2*n, name = "ciphertext")
    final = QuantumCircuit(left1, right1, ancilla1, ancilla2, ciphertext)

    final.x(left1[0])
    final.x(left1[1])
    final.x(ancilla1[0])
    final.h(ancilla1[0])
    final.barrier()

    final = final.compose(oracle1)

    for i in range(3):
        final.measure(left1[i],ciphertext[i])
        final.measure(right1[i],ciphertext[i+3])

    return final

In [24]:
final = generate_circuit(0,0)
print(final)
backend = Aer.get_backend('qasm_simulator')
result = backend.run(final, shots=10000).result()
counts = result.get_counts()
print(counts)

              ┌───┐      ░                                    ┌───┐          »
     left1_0: ┤ X ├──────░────────■────■────■─────────────────┤ X ├──────────»
              ├───┤      ░        │    │    │                 └─┬─┘┌───┐     »
     left1_1: ┤ X ├──────░───■────┼────■────┼────■──────────────┼──┤ X ├─────»
              └───┘      ░   │    │    │    │    │              │  └─┬─┘┌───┐»
     left1_2: ───────────░───■────■────┼────┼────┼────■─────────┼────┼──┤ X ├»
                         ░ ┌─┴─┐  │    │  ┌─┴─┐  │    │         │    │  └─┬─┘»
    right1_0: ───────────░─┤ X ├──┼────┼──┤ X ├──┼────┼─────────┼────■────■──»
                         ░ └───┘┌─┴─┐  │  └───┘┌─┴─┐  │         │    │    │  »
    right1_1: ───────────░──────┤ X ├──┼───────┤ X ├──┼─────────■────┼────■──»
                         ░      └───┘┌─┴─┐     └───┘┌─┴─┐┌───┐  │    │       »
    right1_2: ───────────░───────────┤ X ├──────────┤ X ├┤ X ├──■────■───────»
              ┌───┐┌───┐ ░           └───┘          

# Testing Forrelation Circuit
Function is Boolean and maps strings with even number of 1s to 1, and odd number to -1

In [25]:
qc = QuantumCircuit(5,4)

qc.x(4)
qc.h(4)
qc.barrier()

for i in range(4):
    qc.h(i)
qc.barrier()

for i in range(4):
    qc.cx(i,4)
qc.barrier()


for i in range(4):
    qc.h(i)
qc.barrier()

for i in range(4):
    qc.cx(i,4)
qc.barrier()

for i in range(4):
    qc.h(i)
qc.barrier()

for i in range(4):
    qc.measure(i,i)

print(qc)

                ░ ┌───┐ ░                      ░ ┌───┐ ░                     »
q_0: ───────────░─┤ H ├─░───■──────────────────░─┤ H ├─░───■─────────────────»
                ░ ├───┤ ░   │                  ░ ├───┤ ░   │                 »
q_1: ───────────░─┤ H ├─░───┼────■─────────────░─┤ H ├─░───┼────■────────────»
                ░ ├───┤ ░   │    │             ░ ├───┤ ░   │    │            »
q_2: ───────────░─┤ H ├─░───┼────┼────■────────░─┤ H ├─░───┼────┼────■───────»
                ░ ├───┤ ░   │    │    │        ░ ├───┤ ░   │    │    │       »
q_3: ───────────░─┤ H ├─░───┼────┼────┼────■───░─┤ H ├─░───┼────┼────┼────■──»
     ┌───┐┌───┐ ░ └───┘ ░ ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐ ░ └───┘ ░ ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐»
q_4: ┤ X ├┤ H ├─░───────░─┤ X ├┤ X ├┤ X ├┤ X ├─░───────░─┤ X ├┤ X ├┤ X ├┤ X ├»
     └───┘└───┘ ░       ░ └───┘└───┘└───┘└───┘ ░       ░ └───┘└───┘└───┘└───┘»
c: 4/════════════════════════════════════════════════════════════════════════»
                                                    

In [26]:
backend = Aer.get_backend('qasm_simulator')
result = backend.run(qc, shots=10000).result()
counts = result.get_counts()
print((counts["0000"]/10000)**(0.5))

0.25495097567963926


# Cross-Correlation Circuit

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def generate_circuit(a,b):
    
    rounds = 4
    m = 2
    n = 3
    
    round_keys = [[0 for _ in range(n)] for _ in range(m)]
    
    key = [1,1,0,0,0,1]
    
    constant = [0,0,1]
    
    plaintext = [0,1,1,1,0,1]
    
    input_function = QuantumRegister(2*n,name = "input_function")
    
    left1 = QuantumRegister(n,name = "left1")
    right1 = QuantumRegister(n, name = "right1")
    ancilla1 = QuantumRegister(1, name = "ancilla1")
    
    ciphertext = ClassicalRegister(4*n, name = "ciphertext")
    
    qc = QuantumCircuit(input_function, left1, right1, ancilla1, ciphertext)
    
    # for i in range(3):
    #     if plaintext[i] == 1:
    #         qc.x(left1[i])
    #         qc.x(left2[i])
    
    # for i in range(3):
    #     if plaintext[3 + i] == 1:
    #         qc.x(right1[i])
    #         qc.x(right2[i])
    
    rounds = 4
    m = 2
    n = 3
    
    round_keys = [[0 for _ in range(n)] for _ in range(m)]
    
    key = [1,1,0,0,0,1]
    
    constant = [0,0,1]
    
    for i in range(rounds):
    
        key_index = i%m
    
        if i < m:
            for j in range(0,n):
                round_keys[key_index][j] = key[n*(m - i - 1) + j]
    
        else:
            # print("===============")
            # print(round_keys[key_index])
            for j in range(n):
                round_keys[key_index][j] ^= constant[j] ^ round_keys[key_index - 1][j - 1] ^ round_keys[key_index - 1][j - 2]
            #     print(round_keys[key_index])
            # print("===============")
    
        #print(round_keys[key_index])
        
        if i%2 == 0:
            for j in range(0,n):
                qc.ccx(left1[(j + 1)%n], left1[(j + 2)%n], right1[j])
            for j in range(0,n):
                qc.cx(left1[(j)%n], right1[j])
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    qc.x(right1[j])
        else:
            for j in range(0,n):
                qc.ccx(right1[(j + 1)%n], right1[(j + 2)%n], left1[j])
            for j in range(0,n):
                qc.cx(right1[(j)%n], left1[j])
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    qc.x(left1[j])    
    
    ##########################
    simon_dagger = qc.inverse()
    simon = qc.copy()
    simon_dagger2 = simon_dagger.copy()
    
    ##########################

    if (a < 3):
        qc.cx(left1[a], ancilla1[0])
    else:
        qc.cx(right1[a - 3], ancilla1[0])

    if (b < 3):
        simon.cx(left1[b], ancilla1[0])
    else:
        simon.cx(right1[b - 3], ancilla1[0])
    
    ##########creating oracles###########
    oracle1 = qc.compose(simon_dagger)
    oracle2 = simon.compose(simon_dagger2)
    
    left1 = QuantumRegister(n,name = "left1")
    right1 = QuantumRegister(n, name = "right1")
    ancilla1 = QuantumRegister(1, name = "ancilla1")
    ancilla2 = QuantumRegister(1, name = "ancilla2")
    ciphertext = ClassicalRegister(4*n, name = "ciphertext")
    input_function = QuantumRegister(2*n,name = "input_function")
    final = QuantumCircuit(input_function, left1, right1, ancilla1, ancilla2, ciphertext)


    for i in range(6):
        final.h(input_function[i])
    
    final.x(ancilla1[0])
    final.h(ancilla1[0])
    final.barrier
    
    for i in range(3):
        final.h(left1[i])
        final.h(right1[i])
    
    final.barrier()
    
    final = final.compose(oracle1)
    
    final.barrier()
    
    for i in range(3):
        final.h(left1[i])
        final.h(right1[i])
    
    final.barrier()
    
    for i in range(3):
        final.ccx(input_function[i],left1[i],ancilla1[0])
        final.ccx(input_function[i+3],right1[i],ancilla1[0])
    
    final.barrier()
    
    for i in range(3):
        final.h(left1[i])
        final.h(right1[i])
    
    final.barrier()
    
    final = final.compose(oracle2)
    
    final.barrier()
    
    for i in range(3):
        final.h(left1[i])
        final.h(right1[i])
    
    qc.barrier()
    
    for i in range(3):
        final.measure(left1[i],ciphertext[i])
        final.measure(right1[i],ciphertext[i+3])

    for i in range(6):
        final.measure(input_function[i],ciphertext[i+6])
    #Last six are for the custom input function
    #first six are for cross correlation
    #classical channels indexes 1->12 go from right->left
    #print(final)
    return final

In [4]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
for i in range(6):
    for j in range(i+1,6):
        print(i , j)
        final = generate_circuit(i,j)
        #print(final)
        backend = Aer.get_backend('qasm_simulator')
        #result = backend.run(final, shots=1000).result()
        result = backend.run(final, shots=1000000).result()
        counts = result.get_counts()
        selected_bitstrings = {bs: freq for bs, freq in counts.items() if bs == '111111000000'}
        for k in selected_bitstrings:
            if k == "000000000000": print("correlation found")
        sum1 = 0
        for k in selected_bitstrings:
            print(k , ((counts[k]/1000000)**(0.5))* (2**9))
        #     sum1 += (((counts[k]/1000000)**(0.5))* (2**9))**2
        # print("total = ", sum1, " || bounded by = ",2**12, "and", 2**18)
        # print("============================")
        # try:
        #     print(counts)
        # except KeyError:
        #     print(counts)
        #print(counts)

0 1
111111000000 24.24848432376754
0 2
0 3
111111000000 11.875751428857038
0 4
0 5
111111000000 12.105308587557774
1 2
1 3
1 4
111111000000 12.180866307451208
1 5
111111000000 11.85365698845719
2 3
2 4
2 5
111111000000 8.143858544940477
3 4
111111000000 23.762537238266457
3 5
4 5


# Classical Reduced Simon

In [7]:
def classical_simon(inp,key = [1,1,0,0,0,1]):
    rounds = 4
    m = 2
    n = 3
    
    round_keys = [[0 for _ in range(n)] for _ in range(m)]
    
    # left1 = [0,1,1]
    # right1 = [1,0,1]

    left1 = inp[:3]
    right1 = inp[3:]
    
    #key = [1,1,0,0,0,1]
    
    constant = [0,0,1]
    
    
    for i in range(rounds):
        
        key_index = i%m
    
        if i < m:
            for j in range(0,n):
                round_keys[key_index][j] = key[n*(m - i - 1) + j]
    
        else:
            # print("===============")
            # print(round_keys[key_index])
            for j in range(n):
                round_keys[key_index][j] ^= constant[j] ^ round_keys[key_index - 1][j - 1] ^ round_keys[key_index - 1][j - 2]
            #     print(round_keys[key_index])
            # print("===============")
    
        #print("key used:", round_keys[key_index])
        
        if i%2 == 0:
            for j in range(0,n):
                #left1[(j + 1)%n] ^=  left1[(j + 2)%n] ^ right1[j]
                right1[j] ^=  left1[(j + 2)%n] & left1[(j + 1)%n]
            for j in range(0,n):
                right1[j] ^= left1[(j)%n]
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    right1[j] ^= 1
        else:
            for j in range(0,n):
                #right1[(j + 1)%n] ^= right1[(j + 2)%n] ^ left1[j]
                left1[j] ^= right1[(j + 1)%n] & right1[(j + 2)%n]
            for j in range(0,n):
                left1[j] ^= right1[(j)%n]
            for j in range(0,n):
                if round_keys[key_index][j] == 1:
                    left1[j] ^= 1  
    
        
        #print(left1,right1)
    return left1 + right1
        #print("==================")
        

In [8]:
#print(classical_simon(plaintext,[1,1,0,0,0,1]))
def generate_binary_array(n):
    # Convert integer to binary and remove '0b' prefix
    binary = bin(n)[2:]
    # Pad with zeros to make it length 6
    binary = binary.zfill(6)
    # Convert string to array of integers
    return [int(bit) for bit in binary]

# Example usage: generate arrays from 0 to 63 (2^6 - 1)
# for i in range(64):
#     array = generate_binary_array(i)
#     print(array)

In [16]:
import numpy as np
grid = np.zeros((6, 6), dtype=int)

set_string = [1,1,1,1,1,1]

for i in range(64):
    inp = generate_binary_array(i)
    temp = [(inp[j] + set_string[j])%2 for j in range(6)]
    #print(inp,temp)
    output1 = classical_simon(inp)
    output2 = classical_simon(temp)
    for i in range(6):
        for j in range(6):
            if (output1[i] == output2[j]):
                grid[i][j] += 1
            else:
                grid[i][j] -= 1

for i in range(6):
    for j in range(i+1,6):
        if (grid[i][j] != 0):
            print(i,j)
            print(np.abs(grid[i][j]))            

0 1
24
0 3
12
0 5
12
1 4
12
1 5
12
2 5
8
3 4
24


In [15]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
for i in range(6):
    for j in range(i+1,6):
        final = generate_circuit(i,j)
        #print(final)
        backend = Aer.get_backend('qasm_simulator')
        #result = backend.run(final, shots=1000).result()
        result = backend.run(final, shots=1000000).result()
        counts = result.get_counts()
        selected_bitstrings = {bs: freq for bs, freq in counts.items() if bs == '111111000000'}
        for k in selected_bitstrings:
            if k == "000000000000": print("correlation found")
        sum1 = 0
        if len(selected_bitstrings) == 0:continue
        for k in selected_bitstrings:
            print(i , j)
            print(((counts[k]/1000000)**(0.5))* (2**9))
        #     sum1 += (((counts[k]/1000000)**(0.5))* (2**9))**2
        # print("total = ", sum1, " || bounded by = ",2**12, "and", 2**18)
        # print("============================")
        # try:
        #     print(counts)
        # except KeyError:
        #     print(counts)
        #print(counts)

0 1
23.696253881151762
0 3
11.742561219768028
0 5
11.764864300109883
1 4
11.875751428857038
1 5
11.73139377908695
2 5
7.731030461717248
3 4
23.79560967909837
